# OBIS full export

Sometimes you need to do analyses using big chunks of OBIS data. In those cases, downloading it through the API is not practical. OBIS also offer the full export of all OBIS occurrence data in `parquet` or `csv` formats.

You can download the full export here (~15GB): https://obis.org/data/access/

We will work with the `parquet` file. [Parquet](https://parquet.apache.org/) is a column-based performant and light format developed by Apache. We can work with `parquet` files on R and Python using the `arrow` [package.](https://arrow.apache.org/docs/r/index.html). You can learn more about parquet in [this tutorial](https://resources.obis.org/tutorials/arrow-obis/).

Loading all this data at once in your computer is usually impossible, as it will not fit in memory. But that is not a problem! Arrow offer us many ways of loading just the structure of the file and perform filters without loading the dataset in memory. Let's explore how to do that.

<span style="color: orange;">NOTE: instead of downloading all the data, we will use the S3 file in this practical. However, it is an older copy. For the most up-to-date always download from the website.</span>

You may need to install `arrow` and `h3jsr`. If that is the case, before calling `install.packages()` run those two lines:

```
install.packages("bspm")
suppressMessages(bspm::enable())
```

In [ ]:
library(arrow)
library(dplyr)
library(ggplot2)

ds_path <- "s3://obis-products/exports/obis_20230726.parquet"

ds <- open_dataset(ds_path)

ds

We call `arrow::open_dataset` to open a file (or a folder with multiple files) without loading it into memory. Note that you can just see the columns and data types.

As a first example, we will select only the columns `scientificName`, `family`, and `date_year`. Then, we will filter by the family **Pomacentridae**. Finally, we will group by `scientificName` and `date_year` and count the number of records:

In [ ]:
pomac_records <- ds |> 
    select(scientificName, family, date_year) |> 
    filter(family == "Pomacentridae") |> 
    group_by(scientificName, date_year) |> 
    count() |> 
    collect()

head(pomac_records)

Ok, now let's get the records for the "Nemo"s species, and plot in a map.

In [ ]:
sp_nemos <- robis::checklist("Amphiprion")

sp_nemos <- sp_nemos[sp_nemos$taxonRank == "Species",]

sp_nemo_top4 <- sp_nemos[order(sp_nemos$records, decreasing = T), ][1:4,]

nemo_data <- ds |> 
    select(scientificName, decimalLongitude, decimalLatitude) |> 
    filter(scientificName %in% sp_nemo_top4$scientificName) |> 
    collect()

wrld <- rnaturalearth::ne_countries(returnclass = "sf")

ggplot(nemo_data) +
    geom_sf(data = wrld) +
    geom_point(aes(x = decimalLongitude, y = decimalLatitude, color = scientificName)) +
    facet_wrap(~ scientificName) +
    theme_light()


Although this works fine, for big queries it can be a bit slow, even if the file is local. Another approach that is very good is [`duckDB`](https://duckdb.org/docs/api/r.html). It not only enable to do the same queries, using SQL, but it also have a [spatial extension.](https://duckdb.org/docs/extensions/spatial/functions)

In [ ]:
library(DBI)
library(duckdb)
library(glue)

con <- dbConnect(duckdb())
dbSendQuery(con, "install spatial; load spatial;")
dbSendQuery(con, "install httpfs; load httpfs;")

query <- glue(
    "
    select scientificName, count(*) as number_records
    from read_parquet('{ds_path}')
    where family = 'Pomacentridae'
    group by scientificName
    "
)

pom_fam <- dbGetQuery(con, query)

pom_fam

And for spatial queries is also straightforward. We just need to read the longitude and latitude as Spatial points and then use ST_Intersects to intersect with our geometry (the WKT).

In [ ]:

# And for the case of an area is much faster because it supports spatial queries!
wkt_area <- "POLYGON ((-79.189453 27.293689, -79.584961 23.765237, -75.9375 22.43134, -73.959961 24.726875, -74.750977 27.176469, -79.189453 27.293689))"

query <- glue::glue(
    "
  select distinct scientificName
  from read_parquet('{ds_path}')
  where ST_Intersects(ST_Point(decimalLongitude, decimalLatitude), ST_GeomFromText('{wkt_area}'))
  "
)

all_sps_area <- dbGetQuery(con, query)

head(all_sps_area)